In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# Any results you write to the current directory are saved as output.
data = pd.read_csv("../input/train.csv")
print(data.head())
print(data.shape)

   Id  MSSubClass MSZoning    ...     SaleType  SaleCondition SalePrice
0   1          60       RL    ...           WD         Normal    208500
1   2          20       RL    ...           WD         Normal    181500
2   3          60       RL    ...           WD         Normal    223500
3   4          70       RL    ...           WD        Abnorml    140000
4   5          60       RL    ...           WD         Normal    250000

[5 rows x 81 columns]
(1460, 81)


** Let's check missing values **

In [3]:
print(data.isnull().sum())
columns_missing_values = [c for c in list(data.columns) if data[c].isnull().any()]
print(columns_missing_values)

Id                  0
MSSubClass          0
MSZoning            0
LotFrontage       259
LotArea             0
Street              0
Alley            1369
LotShape            0
LandContour         0
Utilities           0
LotConfig           0
LandSlope           0
Neighborhood        0
Condition1          0
Condition2          0
BldgType            0
HouseStyle          0
OverallQual         0
OverallCond         0
YearBuilt           0
YearRemodAdd        0
RoofStyle           0
RoofMatl            0
Exterior1st         0
Exterior2nd         0
MasVnrType          8
MasVnrArea          8
ExterQual           0
ExterCond           0
Foundation          0
                 ... 
BedroomAbvGr        0
KitchenAbvGr        0
KitchenQual         0
TotRmsAbvGrd        0
Functional          0
Fireplaces          0
FireplaceQu       690
GarageType         81
GarageYrBlt        81
GarageFinish       81
GarageCars          0
GarageArea          0
GarageQual         81
GarageCond         81
PavedDrive

In [19]:
# Create objects for target and predictors.
y = data['SalePrice'].values
temp_X = data.drop(['SalePrice'], axis=1)
X = temp_X.select_dtypes(exclude=['object']).values
print(X)

[[1.000e+00 6.000e+01 6.500e+01 ... 0.000e+00 2.000e+00 2.008e+03]
 [2.000e+00 2.000e+01 8.000e+01 ... 0.000e+00 5.000e+00 2.007e+03]
 [3.000e+00 6.000e+01 6.800e+01 ... 0.000e+00 9.000e+00 2.008e+03]
 ...
 [1.458e+03 7.000e+01 6.600e+01 ... 2.500e+03 5.000e+00 2.010e+03]
 [1.459e+03 2.000e+01 6.800e+01 ... 0.000e+00 4.000e+00 2.010e+03]
 [1.460e+03 2.000e+01 7.500e+01 ... 0.000e+00 6.000e+00 2.008e+03]]


** Let's check missing values within the numerical predictors ** 
#### We kept numerical predictors only for simplicity.

** Okay seems like we have only two columns with missing values. **

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

# Impute missing data.
imputer = Imputer()
X = imputer.fit_transform(X)

# Split data into train and test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [22]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

rg = RandomForestRegressor()
rg.fit(X_train, y_train)
y_pred = rg.predict(X_test)
print('Mean absolute error: ', mean_absolute_error(y_test, y_pred))




Mean absolute error:  20337.71729452055


### In the following section we will use XGBoost and compare it's accuracy to our previous RandomForest regressor. Then along we will tune our Hyperparameters.

In [24]:
from xgboost import XGBRegressor

# Instantiate the XGBoost regressor.
my_model = XGBRegressor()

my_model.fit(X_train, y_train, verbose=False)
predictions = my_model.predict(X_test)
print('Error with XGBoost: ', mean_absolute_error(y_test, predictions))

Error with XGBoost:  17740.013484589042


** We can clearly see here that the error dropped considerably **
** Let's fine tune now some hyperparameters for more accuracy **